In [ ]:
import datetime
import os

import pandas as pd
from pandera.typing import DataFrame

from usskscadaocclient import ScadaConnector

from scrap_core.heat_matching import find_matching, AppData, DbData

from experimental.analyses.common import setup_django, setup_env

In [ ]:
setup_env()

In [ ]:
setup_django()

from scrap.models import ClosedHeatV2
from scrap.tasksimpl.evaluation import closed_heats_to_df, get_db_data, scada_data_to_df

In [ ]:
def sort_dict_by_keys(d):
    return {key: d[key] for key in sorted(d)}


def get_app_data(dt_from: datetime.datetime, dt_to: datetime.datetime) -> DataFrame[AppData]:
    closed_heats = ClosedHeatV2.objects.filter(
        created_at__gte=dt_from,
        created_at__lte=dt_to,
    )

    df_tmp = closed_heats_to_df(closed_heats)

    df_tmp[AppData.scrap_charge] = df_tmp[AppData.scrap_charge].apply(sort_dict_by_keys)

    return df_tmp

In [ ]:
SCADA_DB = {
    "host": os.environ.get("SCADA_DB_HOST"),
    "port": os.environ.get("SCADA_DB_PORT"),
    "database": os.environ.get("SCADA_DB_SID"),
    "username": os.environ.get("SCADA_DB_USER"),
    "password": os.environ.get("SCADA_DB_PASSWORD"),
}

my_dt_from = datetime.datetime(2023, 5, 15)
my_dt_to = datetime.datetime(2023, 5, 16)

my_time_tolerance = datetime.timedelta(hours=24)

In [ ]:
df_data_app = get_app_data(my_dt_from - my_time_tolerance, my_dt_to)

df_data_app["heat_key"] = None

df_data_app.head(20)

In [ ]:
df_data_app.to_csv("data_app.csv")

## Db Data

In [ ]:
data_db = get_db_data(my_dt_from, my_dt_to + my_time_tolerance, ScadaConnector(conf=SCADA_DB))

df_data_db = scada_data_to_df(data_db)

df_data_db[DbData.scrap_charge] = df_data_db[DbData.scrap_charge].apply(sort_dict_by_keys)

df_data_db.head(20)

In [ ]:
m = find_matching(df_data_app, df_data_db, my_time_tolerance)

### Matched heats

In [ ]:
df_data_app["heat_key"] = df_data_app["heat_key"].combine_first(pd.Series(m))

df_data_app.head()

In [ ]:
df_all = pd.merge(df_data_app, df_data_db, on="heat_key", how="left", suffixes=("_app", "_db"))

df_all.head()

In [ ]:
df_all.sort_values("heat_datetime", ascending=False)

In [ ]:
df_sel = df_all[
    # (my_dt_from.date() - datetime.timedelta(hours=24) <= df_all["heat_datetime"].dt.date) &
    # (df_all["heat_datetime"].dt.date <= my_dt_to.date()) &
    (df_all["steelshop_app"] == 2)
    # & (df_all["baskets_app"] == (91, 102))
].copy()

df_sel[["heat_key", "baskets_app", "baskets_db", "created_at", "heat_datetime", "scrap_charge_app", "scrap_charge_db"]].sort_values("heat_datetime", ascending=False)

### App unmatched heats

In [ ]:
df_data_app[df_data_app["heat_key"].isna()].sort_values("created_at").head(50)

### Db unmatched heats

In [ ]:
df_data_db[~df_data_db["heat_key"].isin(df_data_app["heat_key"])].sort_values("heat_datetime").head(50)